In [28]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import string

from keras import layers
from keras import losses
import keras

from keras.layers import TextVectorization

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import NMF

from sklearn.metrics import mean_squared_error

In [33]:
m = np.array([
    [0, 1, 0, 1, 2, 2, 0, 4] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 1, 1, 3, 4, 1, 2] + [round(np.random.rand(),3) for _ in range(10)],
    [2, 3, 1, 1, 2, 2, 2, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [1, 1, 1, 0, 1, 1, 4, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 2, 3, 4, 1, 1, 2, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 0, 0, 0, 1, 0, 3, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 0, 1, 2, 2, 4, 4] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 0, 1, 2, 2, 3, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 1, 1, 3, 4, 3, 4] + [round(np.random.rand(),3) for _ in range(10)],
    [2, 3, 1, 1, 2, 2, 4, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [1, 1, 1, 0, 1, 1, 3, 2] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 2, 3, 4, 1, 1, 2, 3] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 0, 0, 0, 1, 0, 1, 0] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 0, 1, 2, 2, 3, 0] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 0, 1, 2, 2, 0, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 1, 1, 3, 4, 2, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [2, 3, 1, 1, 2, 2, 1, 4] + [round(np.random.rand(),3) for _ in range(10)],
    [1, 1, 1, 0, 1, 1, 3, 2] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 2, 3, 4, 1, 1, 4, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 0, 0, 0, 1, 0, 3, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 0, 1, 2, 2, 0, 2] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 0, 1, 2, 2, 1, 4] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 1, 1, 3, 4, 0, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [2, 3, 1, 1, 2, 2, 3, 0] + [round(np.random.rand(),3) for _ in range(10)],
    [1, 1, 1, 0, 1, 1, 1, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 2, 3, 4, 1, 1, 3, 4] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 0, 0, 0, 1, 0, 4, 1] + [round(np.random.rand(),3) for _ in range(10)],
    [0, 1, 0, 1, 2, 2, 4, 3] + [round(np.random.rand(),3) for _ in range(10)]
])

for row in m:
    row[:8] = np.random.randint(0, 2, size=8)

# I've initialized the first half of the columns as 1's and 0's to mimic how the csv file will have 1's 
# for when the book can be found in that genre and 0's for when that book is not found in that genre
# the second half of the columns I've given random floats to mimic the percentages that we'll gather from 
# the open library reviews

In [34]:
# I've listed about 30 of the keywords from reviews

reviews = [
    'Fast paced', 'Medium paced', 'Slow paced', 'Meandering','Engaging', 'Exciting', 'Neutral', 
    'Boring', 'Dense', 'Clearly written', 'Succinct', 'Effective explanations', 
    'Well organized', 'Beginner', 'Intermediate', 'Advanced', 'Layman', 
    'Comprehensive', 'Focused', 'Horror', 'Romance', 'Mystery', 'Fiction', 'Drama', 'Fantasy', 'Humor', 
    'Review', 'Classic', 'Satire', 'Exploratory', 'Philisophical', 'Crime', 'Textbook', 'Suspenseful', 
    'Romantic', 'Emotional', 'Scientific', 'Tense', 'Hopeful',
    'Dark', 'Sad', 'Strange', 'Mysterious', 'Informative', 
    'Fearful', 'Angry', 'Highly recommend', 'Recommend', 
    'Life changing', 'Quotable', 'Underrated', 'Medium', 'Long', 'Short',
    'Trendy', 'Accurate', 'Biased', 'Adult themes', 
    'Trigger warnings', 'Offensive language', 'Olde', 'Slang', 
    'Entertainment', 'Broaden perspective', 'Inspiration']

In [35]:
# the columns of the csv will start with the genres/ subjects
# then I've added the first five of the keywords from reviews to create all of our columns

dataset = pd.DataFrame(m, columns = ["Romance", "Fantasy", "Historical Fiction", "Horror", "Humor", "Literature", 
                 "Mystery/Thriller", "Science Fiction"] + reviews[:10])

# the rows of the csv file will have book titles
dataset.index = [
    "The Great Gatsby", "To Kill a Mockingbird", "1984", "Pride and Prejudice", "Moby-Dick", "War and Peace", "The Catcher in the Rye",
    "The Hobbit", "Ulysses", "The Odyssey", "Crime and Punishment", "Jane Eyre", "Brave New World", "Wuthering Heights", "The Divine Comedy",
    "The Brothers Karamazov",  "Anna Karenina",  "Madame Bovary",  "The Iliad",  "Don Quixote",  "One Hundred Years of Solitude",  "The Sound and the Fury",
    "Invisible Man", "Beloved", "Mrs. Dalloway", "Slaughterhouse-Five", "The Sun Also Rises",
    "Middlemarch"]


# V is our csv file represented as a matrix to mimic: V = W X H
V = dataset

# to make it easier to see, I made our csv file into a pandas dataframe
df = pd.DataFrame(dataset)
df

,Romance,Fantasy,Historical Fiction,Horror,Humor,Literature,Mystery/Thriller,Science Fiction,Fast paced,Medium paced,Slow paced,Meandering,Engaging,Exciting,Neutral,Boring,Dense,Clearly written
The Great Gatsby,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.241,0.716,0.294,0.289,0.199,0.803,0.205,0.418,0.431,0.731
To Kill a Mockingbird,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.256,0.710,0.356,0.807,0.209,0.776,0.262,0.718,0.070,0.423
1984,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.972,0.527,0.052,0.048,0.672,0.148,0.046,0.584,0.164,0.080
Pride and Prejudice,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.994,0.561,0.089,0.068,0.582,0.033,0.694,0.324,0.101,0.634
Moby-Dick,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.780,0.012,0.851,0.548,0.707,0.285,0.505,0.079,0.806,0.052
War and Peace,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.983,0.753,0.505,0.858,0.501,0.479,0.989,0.623,0.977,0.762
The Catcher in the Rye,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.192,0.255,0.575,0.971,0.404,0.977,0.932,0.856,0.496,0.391
The Hobbit,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.160,0.186,0.984,0.128,0.434,0.556,0.713,0.534,0.314,0.656
Ulysses,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.777,0.278,0.565,0.036,0.160,0.208,0.838,0.284,0.336,0.842
The Odyssey,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.148,0.778,0.029,0.114,0.374,0.304,0.466,0.597,0.366,0.792


In [39]:
# def rank_calculation(data=V):
#     """
#     Calculate the optimal rank of the specified dataframe. USE THIS TO FIND RANK (# OF CLUSTERS) THEN DELETE AND HARDCODE AFTER!!!
#     """
#     # Read the data
#     df = data
    
#     # Calculate benchmark value
#     benchmark = np.linalg.norm(df, ord='fro') * 0.0001
    
#     # Iterate through various values of rank to find optimal
#     rank = 3
#     while True:
        
#         # initialize the model
#         model = NMF(n_components=rank, init='random', random_state=0, max_iter=5000)
#         W = model.fit_transform(df)
#         H = model.components_
#         V = W @ H
        
#         # Calculate RMSE of original df and new V
#         RMSE = np.sqrt(mean_squared_error(df, V))
        
#         if RMSE < benchmark:
#             return rank, V
        
#         # Increment rank if RMSE isn't smaller than the benchmark
#         rank += 1

#     return rank

# # Hardcode the value so that we don't have to run the above function
# optimal_rank = rank_calculation()
# # optimal_rank = 15 
# optimal_rank

In [50]:
# to estimate the possible shared "topics", I've created a list of 5 topics
topics = ['Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5', 
    'Topic 6', 'Topic 7', 'Topic 8', 'Topic 9', 'Topic 10',
    'Topic 11', 'Topic 12', 'Topic 13', 'Topic 14', 'Topic 15',
    'Topic 16', 'Topic 17', 'Topic 18', 'Topic 19', 'Topic 20',
    'Topic 21', 'Topic 22', 'Topic 23', 'Topic 24', 'Topic 25',
    'Topic 26', 'Topic 27'] # OPTIMAL = 27 FOR THIS EXAMPLE

# here we're able to define the model using the NMF package which we'll use to decompose W and H
model = NMF(n_components=27, init='random', random_state=0, max_iter = 3000) 

In [51]:
# creating W from V by taking in the rows of V and the columns from topics
W = model.fit_transform(V, 500)

# creating the other matrix H based on the topics and columns of V
H = pd.DataFrame(model.components_)

# Renaming the indices of H to reflect the topics
H.index = topics

# creating a new numpy array W_df to reflect the dataframe, not the model matrix
# !!!!!!!!!! I rounded to two decimal places just so I can see the data better right now !!!!!!!!!!
# also ensures the columns of W match the rows of H
W_df = pd.DataFrame(np.round(W, 2), columns = H.index)
W_df

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,...,Topic 18,Topic 19,Topic 20,Topic 21,Topic 22,Topic 23,Topic 24,Topic 25,Topic 26,Topic 27
0,0.00,0.00,0.00,1.47,1.12,0.00,0.00,0.07,0.00,0.00,...,0.58,0.08,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.00
1,0.00,0.00,0.50,0.00,0.59,0.00,0.00,0.07,0.00,0.00,...,0.00,0.00,0.00,0.00,0.52,0.06,0.00,0.00,0.00,0.00
2,0.00,0.07,0.00,0.03,0.01,0.00,0.00,0.00,0.00,0.00,...,1.36,0.00,0.00,0.00,0.00,0.01,0.00,0.95,0.00,0.00
3,0.00,0.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.01,0.00,0.00,0.00,0.00,0.00,1.01,0.00,0.13
4,1.76,0.00,0.00,0.00,0.00,0.00,0.80,0.00,0.00,0.41,...,0.00,0.60,0.01,0.00,0.00,0.06,0.01,0.00,0.00,0.06
5,0.00,0.00,0.00,1.27,0.03,0.00,0.00,0.17,0.00,0.00,...,1.01,0.44,0.13,0.06,0.00,0.02,0.00,0.00,0.00,0.59
6,1.23,1.25,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.86,...,0.30,0.19,0.15,0.00,0.00,0.00,0.08,0.00,0.50,0.00
7,2.58,1.26,0.02,0.00,0.00,0.27,0.00,0.00,0.00,0.00,...,0.00,0.32,0.00,0.00,0.76,0.00,0.49,0.00,0.00,0.00
8,0.32,1.26,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.65,...,0.00,0.35,0.00,0.00,0.00,0.59,0.42,0.00,0.28,0.00
9,0.00,0.00,0.00,1.13,0.05,0.00,0.00,0.05,0.00,0.00,...,1.18,0.00,0.00,0.00,0.00,0.00,0.47,0.00,0.56,0.00


In [52]:
# basically repeating everything from W but respectively for H
H.index = topics
H.columns = V.columns
H_df = pd.DataFrame(np.round(H,2))
H_df

,Romance,Fantasy,Historical Fiction,Horror,Humor,Literature,Mystery/Thriller,Science Fiction,Fast paced,Medium paced,Slow paced,Meandering,Engaging,Exciting,Neutral,Boring,Dense,Clearly written
Topic 1,0.00,0.00,0.00,0.21,0.00,0.17,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Topic 2,0.00,0.00,0.00,0.00,0.00,0.00,0.80,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,0.00,0.00
Topic 3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.86,0.00,0.26,0.00,0.18,0.16,0.12,0.00,0.00,0.03,0.00
Topic 4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.42,0.00,0.00,0.08,0.00,0.08,0.00,0.19,0.00
Topic 5,0.00,0.38,0.00,0.00,0.00,0.00,0.00,0.00,0.11,0.00,0.20,0.16,0.00,0.70,0.00,0.25,0.05,0.65
Topic 6,0.00,0.00,0.00,0.53,0.00,0.00,0.00,0.53,0.00,0.00,0.23,0.28,0.34,0.00,0.02,0.30,0.35,0.05
Topic 7,0.86,0.00,0.59,0.78,0.00,0.70,0.00,0.00,0.77,0.00,0.30,0.13,0.79,0.00,0.00,0.00,0.26,0.00
Topic 8,0.00,0.56,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.22,0.27,0.17,0.00,0.00
Topic 9,0.78,0.00,0.83,0.00,0.00,0.00,0.00,0.67,0.10,0.39,0.10,0.00,0.13,0.00,0.69,0.18,0.17,0.09
Topic 10,0.00,0.00,1.17,0.00,0.00,0.33,0.00,0.00,0.00,0.00,0.30,0.00,0.02,0.00,0.00,0.00,0.02,0.00


In [53]:
# actually applying NMF again through matrix-multiplication to reconstruct the approximation of V
new_V = W @ H

# rounding decimals again to 5 for easier reading
new_V = pd.DataFrame(np.round(new_V,5), columns=V.columns)
new_V.index = V.index
new_V.index.name = "Title"
new_V

,Romance,Fantasy,Historical Fiction,Horror,Humor,Literature,Mystery/Thriller,Science Fiction,Fast paced,Medium paced,Slow paced,Meandering,Engaging,Exciting,Neutral,Boring,Dense,Clearly written
Title,,,,,,,,,,,,,,,,,,
The Great Gatsby,0.00026,1.00014,0.00083,0.00036,0.00000,0.00067,0.00014,0.00056,0.24096,0.71626,0.29476,0.28937,0.20024,0.80371,0.20528,0.41671,0.42977,0.73035
To Kill a Mockingbird,0.00538,1.00050,0.00000,0.00036,1.00013,0.00217,0.00014,1.00006,0.25584,0.70965,0.35618,0.80702,0.21054,0.77424,0.26277,0.71738,0.07100,0.42483
1984,1.00031,1.00008,0.00083,0.99971,0.00001,0.00053,1.00000,0.00035,0.97187,0.52652,0.05110,0.04933,0.67399,0.14929,0.04847,0.58190,0.16329,0.07936
Pride and Prejudice,0.00138,0.00010,0.99994,1.00005,1.00008,0.00007,1.00005,0.00036,0.99404,0.56131,0.08923,0.06764,0.58111,0.03338,0.69374,0.32437,0.10112,0.63413
Moby-Dick,0.99873,0.00000,1.00033,1.00025,0.00000,0.99952,0.00000,0.00000,0.77990,0.01179,0.85028,0.54732,0.70901,0.28655,0.50680,0.08209,0.80488,0.05253
War and Peace,0.99970,0.99975,0.99978,0.00095,0.99999,0.00106,0.00036,0.00025,0.98292,0.75287,0.50470,0.85805,0.50159,0.47892,0.98919,0.62372,0.97717,0.76225
The Catcher in the Rye,0.99984,0.99989,1.00008,0.99999,0.00003,1.00000,1.00002,0.00019,0.19234,0.25558,0.57477,0.97067,0.40400,0.97740,0.93195,0.85582,0.49628,0.39096
The Hobbit,0.00010,0.00060,0.00027,0.99992,0.99994,0.99997,1.00002,0.99987,0.16034,0.18608,0.98282,0.12722,0.43488,0.55689,0.71336,0.53387,0.31501,0.65611
Ulysses,0.00009,0.00003,1.00003,1.00001,0.99998,1.00009,1.00003,0.00020,0.77702,0.27801,0.56471,0.03818,0.16042,0.20813,0.83821,0.28356,0.33575,0.84194


In [296]:
def top_book_val(book, df):

    # check if the input is in new_V
    if book not in df.index:
        return f'{book} cannot be found in the DataFrame'

    # grabs all the cells for the input book
    book_scores = df.loc[book]

    # sets the largeset value to the first cell
    top_val = book_scores[0]

    # iterate through all the cells to find the largest value and replace top_val
    for i in book_scores:
        if i > top_val:
            top_val = i
    return top_val

test = top_book_val('1984', new_V)
test

1.00031

In [309]:
def find_column(book, df, top_val):
    in_col = [column for column in df.columns if top_val in df[column].values]
    for i in in_col:  # `i` is already a column name
        for j in df.columns:  # Looping through all columns
            for k in df.index:  # Looping through all rows
                # Check if the value matches in the DataFrame
                if df.loc[k,j] == top_val:
                    our_genre = i  # Since `i` is already a column name
                    return our_genre  # Return the genre
    return None 
    
print (find_column('1984', new_V, test))

Romance


In [310]:
def rec_values(book, df, top_val):
    
    # Initialize a list to store the top 5 values
    recs_vals = []

    # Get the row corresponding to the book
    book_row = df.loc[book]
    book_col = find_column(book, df, top_val)
    col_index = df.columns.get_loc(book_col)

    # Iterate through each value in the row
    # for value in df[book_col]:
    
    for value in range(len(df)):
        if df.iloc[value, col_index] != top_val:
            if len(recs_vals) < 5:
                # Add the value to the list if it has less than 5 elements
                recs_vals.append(df.iloc[value, col_index])
                recs_vals.sort()  # Ensure the list is sorted (smallest to largest)
            elif df.iloc[value, col_index] > recs_vals[0]:
                # Replace the smallest value if the current value is larger
                recs_vals[0] = df.iloc[value, col_index]
                recs_vals.sort()  # Keep the list sorted

    return recs_vals
mine = rec_values('1984', new_V, test)
mine

[1.0001, 1.00015, 1.00019, 1.00049, 1.00094]

In [328]:
def recommendations(book, df, recs_vals):
    rows = []

    for i in df.index:
        if df.loc[i, book_col] in recs_vals:
            rows.append(i)
    return rows
    
recommendations('1984', new_V, mine)

['Anna Karenina',
 'The Iliad',
 'Don Quixote',
 'One Hundred Years of Solitude',
 'Beloved']